In [1]:
# import gymnasium as gym
import json
import numpy as np
from stable_baselines3.common.monitor import Monitor
from envs import BlendEnv, MaskedBlendEnv, flatten_and_track_mappings, reconstruct_dict

In [2]:
from sb3_contrib import MaskablePPO
# from sb3_contrib.common.envs.invalid_actions_env import InvalidActionEnvDiscrete
# from sb3_contrib.common.maskable.evaluation import evaluate_policy
from sb3_contrib.common.maskable.utils import get_action_masks
# This is a drop-in replacement for EvalCallback
# from sb3_contrib.common.maskable.callbacks import MaskableEvalCallback

In [3]:
env = MaskedBlendEnv(v = False)
env = Monitor(env)

In [4]:
def lr_scheduler(p):
    if p > 0.9:
        return 1e-2
    if p > 0.75:
        return 4e-3
    if p > 0.25:
        return 1e-3
    else:
        return 2e-4

In [5]:
policy_kwargs = dict(
    net_arch=[dict(pi=[128]*4, vf=[128]*4)]
)

In [6]:
model = MaskablePPO("MlpPolicy", env, tensorboard_log="./logs", clip_range=0.5, learning_rate=lr_scheduler, policy_kwargs=policy_kwargs, ent_coef=0)

c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL\blendv\lib\site-packages\sb3_contrib\common\maskable\policies.py:78: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


# Modified sb3_contrib code

In [7]:
env.action_space.shape

(36,)

In [8]:
env.act_mask.shape

c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL\blendv\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.act_mask to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.act_mask` for environment variables or `env.get_wrapper_attr('act_mask')` that will search the reminding wrappers.
  logger.warn(


torch.Size([36])

In [9]:
env.action_masks().shape

c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL\blendv\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.action_masks to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.action_masks` for environment variables or `env.get_wrapper_attr('action_masks')` that will search the reminding wrappers.
  logger.warn(


(36, 11)

In [10]:
model.learn(total_timesteps=50000)

KeyboardInterrupt: 

In [11]:
model

In [12]:
obs = env.reset()
obs, obs_dict = obs

In [13]:
model.predict(obs)

(array([ 6,  5,  6,  4,  5,  9,  7,  7,  4,  0,  8,  5,  0,  1, 10,  4,  3,
         4,  7,  7,  8,  5,  3,  9,  9,  4,  9,  7,  3,  5,  9,  1,  2, 10,
         5,  2], dtype=int64),
 None)

In [2]:
import torch as th
from gym.spaces import MultiDiscrete

class CustomEnv:
    def __init__(self):
        self.action_dims = [51] * 36
        self.action_space = MultiDiscrete(self.action_dims)

    def process_masks(self, masks):
        if masks is not None:
            # Convert masks to a PyTorch tensor
            print(masks.shape)
            masks_tensor = th.as_tensor(masks, dtype=th.float32)
            # Ensure the masks tensor is correctly shaped
            print(masks_tensor.shape)
            masks_tensor = masks_tensor.view(-1, sum(self.action_dims))
            print(masks_tensor.shape)
            # Then split column-wise for each discrete action dimension
            split_masks = th.split(masks_tensor, self.action_dims, dim=1)  # type: ignore[assignment]
            print(split_masks)
            return split_masks

# Example usage
env = CustomEnv()
masks = th.ones((1, 36, 51))  # Example masks with shape (batch_size, 36, 51)
split_masks = env.process_masks(masks)
# print(split_masks)

torch.Size([1, 36, 51])
torch.Size([1, 36, 51])
torch.Size([1, 1836])
(tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]), tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]), tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]), tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 

In [4]:
th.Tensor([1,2,3])*5

tensor([ 5., 10., 15.])